In [5]:
import mediapipe as mp
import cv2
import numpy as np
from keras.models import model_from_json

emotion_dict = {
    0: "Angry",
    1: "Disgust",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised"
}

# Load emotion detection model
json_file = open('../model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("../model/emotion_model.h5")

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Load video
cap = cv2.VideoCapture(0)

# Process video frames
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        ct=0
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            break

        # Process frame with MediaPipe Face Detection
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = face_detection.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Process each detected face
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, c = image.shape
                x, y, width, height = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)
                face_image = image[y:y+height, x:x+width]

                # Perform emotion detection on the face image
                gray_frame = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                face_roi_gray_frame = cv2.resize(gray_frame, (48, 48))
                cropped_img = np.expand_dims(np.expand_dims(face_roi_gray_frame, -1), 0)

                # Predict emotions
                emotion_prediction = emotion_model.predict(cropped_img)
                maxindex = int(np.argmax(emotion_prediction))
                # if maxindex == 3 or maxindex == 4 or maxindex == 6:
                #     ct+=1
                cv2.putText(image, "Emotion: " + str(emotion_dict[maxindex]), (5,30), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0),
                        1, cv2.LINE_AA)

        # Display the frame in full screen
        cv2.namedWindow('Emotion and Face Detection', cv2.WINDOW_NORMAL)
        cv2.setWindowProperty('Emotion and Face Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('Emotion and Face Detection', image)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 21ms/step
